In [1]:
import yfinance as yf
from finvizfinance.quote import finvizfinance
import pandas as pd
import numpy as np
from datetime import datetime
import re
import requests

In [13]:
def request_historic_financial_data(data_type: str, ticker: str):
  """Funcion para traerse la informacion financiera de la accion
  Información traida desde stockanalysis."""
  # Se intento traer la información desde yahoofinance 
  # pero trae menos historia que stockanalysis
  # ademas yahoofinance no coincide con finchat, stockanalysis si

  # Nos traemos los estados de resultados
  if data_type=="income":
    url = f"https://stockanalysis.com/stocks/{ticker.lower()}/financials"
  elif data_type=="cash_flow":
    url = f"https://stockanalysis.com/stocks/{ticker.lower()}/financials/cash-flow-statement"
  elif data_type=="balance_sheet":
    url = f"https://stockanalysis.com/stocks/{ticker.lower()}/financials/balance-sheet"

  print(url)
  response = requests.get(url)
  if response.status_code==200:
    print("Request successful")

  return response.content


def parse_historic_financial_data(html_data, kpis: list):
  """Funcion para parsear o formatear la info financiera TTM"""

  hist_fin_complete = pd.read_html(html_data)[0]
  hist_fin_complete.columns = [x[1][8:].strip() for x in hist_fin_complete.columns]
  
  # si la rentabilidad bruta no está, ponemos las mismas ventas
  if ("Gross Profit" in kpis)&("Gross Profit" not in hist_fin_complete.iloc[:,0].values):
    print(hist_fin_complete.iloc[:,0].values)
    row_rev = hist_fin_complete[hist_fin_complete.iloc[:,0]=="Revenue"].copy()
    row_rev.iloc[:,0] = "Gross Profit"
    hist_fin_complete = pd.concat([hist_fin_complete, row_rev])
    print("No se encontro el kpi Gross Profit... ajustado al Revenue")

  if "Inventory" in kpis:
    # si no encontramos Inventory en la tabla (Ej: empresas de IT)
    if hist_fin_complete.iloc[:,0].isin(["Inventory"]).sum()==0:
      kpis = [x for x in kpis if "Inventory"!=x]

  # a partir de la tabla completa, nos quedamos con los indicadores
  # que nos interesan, por ejemplo: ventas total, utilidad bruta, etc..
  hist_fin = hist_fin_complete[hist_fin_complete.iloc[:,0].isin(kpis)].copy().T
  hist_fin.columns = hist_fin.iloc[0, :]
  hist_fin = hist_fin.iloc[1:,:]
  hist_fin = hist_fin.reset_index()
  hist_fin = hist_fin.rename(columns={"index": "period"})
  hist_fin = hist_fin[hist_fin[kpis[0]]!="Upgrade"]
  hist_fin["earning_date"] = pd.to_datetime(
    hist_fin["period"], format="%b %d, %Y"
  ).dt.strftime("%Y-%m-%d")
  hist_fin = hist_fin.set_index(["earning_date"])
  hist_fin = hist_fin.drop(columns=["period"])

  # casteo de los indicadores porque vienen tipo object
  for x in kpis:
    hist_fin[x] = hist_fin[x].astype(float)

  return hist_fin_complete, hist_fin


def get_financial_data(ticker: str, data_type: str):
  """Esta funcion extrae los datos financeros de estado de resultados
  y flujo de caja para un ticker dado.
  
  Arguments
  ---------
  ticker (str): ticker de la empresa que se desea analizar
  data_type (str): tipo de datos que se desea extraer, puede ser income o cash_flow
  """
  # extraer de la pagina stockanalysis la tabla ttm historico
  response_data = request_historic_financial_data(data_type=data_type, ticker=ticker)

  # filtrado de la tabla para quedarnos con los indicadores de interes
  if data_type=="income":
    kpis = ["Revenue", "Gross Profit", "Operating Income", "Net Income"]
  elif data_type=="cash_flow":
    kpis = ["Operating Cash Flow", "Free Cash Flow"]
  elif data_type=="balance_sheet":
    kpis = [
      "Cash & Short-Term Investments", 
      "Total Current Assets", 
      "Total Current Liabilities", 
      "Inventory", 
      "Total Debt",
      "Total Assets"
    ]

  data_complete, data = parse_historic_financial_data(response_data, kpis)
  
  return data_complete, data


def score_growth(hist_kpis: pd.DataFrame) -> float:
  """
  Esta funcion construye la proporcion de periodos que ha crecido un kpi para una empresa.
  Esta proporcion la multiplica por 10 para obtener un score de 0 a 10.
  Ejemplo: compara las ventas totales de cada año vs el año anterior correspondiente, calculando
  la proporcion de años que se ha presentado crecimiento

  Arguments
  ---------
  hist_kpis (pd.DataFrame): dataframe periodos x kpi

  Return
  ------
  score (float): score de 0 a 10
  """
  print("=="*20)
  print("Revisando crecimiento de la compañía...")
  # validamos que tengamos al menos tres años de analisis
  if hist_kpis.shape[0]>3:
    # indicador a comparar la empresa con su historia
    results = []
    for kpi in hist_kpis.columns:
      print(f"Analizando kpi: {kpi}")

      # Numero de periodos que el kpi ha crecido
      num_per_growth = sum(hist_kpis[kpi] > hist_kpis[kpi].shift(-1))

      # proporcion de periodos que el kpi ha crecido
      # restamos 1 al denominador porque el primer periodo
      # no lo podemos comparar con un periodo anterior
      prop_per_growth = num_per_growth / (hist_kpis.shape[0]-1)

      # concatenamos para cada kpi, la proporcion de periodos que crecio
      results.append(prop_per_growth)

    # sacamos score del 0 al 10, dandole el mismo peso a cada kpi
    score = 10*sum(results)/len(results)
  else:
    score = None
    print("Sin información suficiente para analizar la empresa")

  print("=="*20)
  print(f"Score de crecimiento: {score}")

  return score


def get_margins_ttm(inc_stmt: pd.DataFrame):
  """Esta función calcula los margenes de los estados de resultados.
  Se calcula a partir de los resultados de los ultimos doce meses (ttm)
  """
  per_compare = inc_stmt.iloc[0, :]
  margins = {
    "Gross M": per_compare["Gross Profit"] / per_compare["Revenue"],
    "Oper M": per_compare["Operating Income"] / per_compare["Revenue"],
    "Profit M": per_compare["Net Income"] / per_compare["Revenue"],
  }
  return margins


def get_competitors_tickers(ticker: str, n_competitors: int):
  """Obtiene los tickers de los competidores asociados a un ticker dado,
  desde la pagina de finviz.com"""

  # Obtener tickers de la competencia
  # Ticker de interes
  stock = finvizfinance(ticker)

  # Obtencion de los competidores segun FinViz
  # basado en la estructura html de la pagina
  attrs = {
    "class": "tab-link",
    "href": re.compile("^screener")
  }
  peers = [
    x for x in stock.soup.findAll("a", attrs)
    if "Peers" in x
  ][0]
  peers = peers.get("href").split("=")[1].split(",")

  # primeros n competidores
  peers = peers[:n_competitors]
  
  return peers


def get_margins_peers(peers: list):
  """Esta funcion calcula los margenes de los estados de resultados para una lista de tickers,
    que son los competidores del ticker analizado"""

  print("Calculando margenes de la competencia...")
  peers_margin = {}
  data_type = "income"
  for tick_peer in peers:
    try:
      _, income_stmt_peer = get_financial_data(tick_peer, data_type)
      peers_margin[tick_peer] = get_margins_ttm(income_stmt_peer)
    except Exception as e:
      print(f"Error ticker: {tick_peer} - {e}")

  return peers_margin


def score_stmt_margins_competitors(
    ticker: str, 
    income_stmt: pd.DataFrame,
    n_competitors: int
  ):
  """Esta función calcula el score de los margenes del estado de resultados
  del ticker de interes vs los competidores.
  El score es la proporcion de indicadores (multiplicado por 10) en los que el ticker de interes
  supera en margenes a los competidores. Todos los indicadores-competidores tienen el mismo peso.

  Arguments:
  ----------
  ticker (str): Ticker de la empresa que se desea analizar.
  income_stmt (pd.DataFrame): Indicadores analizados en el estado de resultados de la empresa de interes.
  n_competitors (int): Cantidad máxima de competidores que se van a extraer para comparar

  Return:
  -------
  score_margins_peers (float): score de 0 a 10
  """
  print(f"Calculando los margenes de la empresa: {ticker}")
  margin_interes = get_margins_ttm(income_stmt)

  print(f"Calculando los margenes de los competidores")
  peers = get_competitors_tickers(ticker, n_competitors)
  print(f"Competidores: {peers}")
  margin_peers = get_margins_peers(peers)

  # se calculara el score de comparacion de margenes
  beat_peers = []
  for _, peer_margin in margin_peers.items():
    for kpi in peer_margin:
      if margin_interes[kpi]>peer_margin[kpi]:
        beat_peers.append(1)
      else:
        beat_peers.append(0)

  score_margins_peers = sum(beat_peers)*10/len(beat_peers)
  print("=="*20)
  print(f"Score comparación vs competencia: {score_margins_peers}")

  return peers, score_margins_peers


def calculate_kpis_balance_general(
    ticker: str,
    income_stmt_complete: pd.DataFrame
):
  """Esta función calcula los indicadores del balance general, como:
  1. Razon corriente
  2. Razón corriente "acida"
  3. Deuda sobre los activos totales
  4. Numero de meses de operación con el dinero en caja
  """
  _, balance = get_financial_data(ticker, "balance_sheet")

  # Dinero en caja que cubra mas de tres meses de operación
  # Total cash and short term investments > selling general & admin expenses, total
  # gastos totales de operacion
  operation_expenses_month = income_stmt_complete.set_index(["nding"]).iloc[:,0]["Selling, General & Admin"]
  operation_expenses_month = float(operation_expenses_month) / 12
  print(f"Gastos operativos mensuales: {round(operation_expenses_month)}")

  # dinero en caja
  cash = balance["Cash & Short-Term Investments"].iloc[0]
  print(f"Dinero en caja: {cash}")

  # meses de operacion que se cubren con la caja
  months_operation = round(cash / operation_expenses_month, 2)
  print(f"Meses de operación con el dinero en caja: {months_operation}")

  # activos corrientes
  current_assets = balance["Total Current Assets"].iloc[0]

  # pasivos corrientes
  current_liabili = balance["Total Current Liabilities"].iloc[0]

  # current ratio>0.7?
  current_ratio = round(current_assets/current_liabili, 2)
  print(f"Current ratio: {current_ratio}")

  # Inventario
  try:
    inventory = balance["Inventory"].iloc[0]
    # quickRatio>0.7?
    quick_ratio = round((current_assets-inventory)/current_liabili, 2)
  except Exception as e:
    quick_ratio = None
    print(e)

  print(f"Quick ratio: {quick_ratio}")

  # Total Debt
  total_debt = balance["Total Debt"].iloc[0]

  # total Assets
  total_assets = balance["Total Assets"].iloc[0]

  # Total Debt / Total Assets
  debt_ratio = round(total_debt/total_assets, 2)
  print(f"Debt ratio: {debt_ratio}")

  kpis_balance = {
      "balance": balance,
      "operation_expenses_month": operation_expenses_month,
      "cash": cash,
      "months_operation": months_operation,
      "current_ratio": current_ratio,
      "quick_ratio": quick_ratio,
      "debt_ratio": debt_ratio
  }
  return kpis_balance


def score_balance_general(kpis_balance: dict):
  
  months_operation = kpis_balance["months_operation"]
  current_ratio = kpis_balance["current_ratio"]
  quick_ratio = kpis_balance["quick_ratio"]
  debt_ratio = kpis_balance["debt_ratio"]

  # reglas de sanidad de una empresa
  kpis_rules = {
      "months_operation": {"var": months_operation, "val": 3},
      "current_ratio": {"var": current_ratio, "val": 0.7},
      "debt_ratio": {"var": debt_ratio, "val": 0.5}
  }
  if quick_ratio:
    kpis_rules["quick_ratio"] = {"var": quick_ratio, "val": 0.7}

  score_balance = []
  for kpiname, info_dict in kpis_rules.items():
    kpi = info_dict["var"]
    rule = info_dict["val"]
    if kpi:
      if kpiname in ["debt_ratio"]:
        score_balance += [1] if kpi<=rule else [0]
      else:
        score_balance += [1] if kpi>=rule else [0]

  score_balance = 10*sum(score_balance)/len(score_balance)

  print("=="*20)
  print(f"Score balance general: {score_balance}")

  return score_balance

In [11]:
ticker = "PLTR"
n_competitors = 5

### Analisis de los estados de resultados de la empresa

In [5]:
data_type = "income"

income_stmt_complete, income_stmt = get_financial_data(ticker, data_type)
print(f"earnings date: {income_stmt.index.values}")

print(f"Vamos a revisar que la empresa {ticker} se encuentre creciendo")
score_stmt_res_growth = score_growth(income_stmt)

https://stockanalysis.com/stocks/pltr/financials
Request successful
earnings date: ['2024-09-30' '2023-12-31' '2022-12-31' '2021-12-31' '2020-12-31'
 '2019-12-31']
Vamos a revisar que la empresa PLTR se encuentre creciendo
Revisando crecimiento de la compañía...
Analizando kpi: Revenue
Analizando kpi: Gross Profit
Analizando kpi: Operating Income
Analizando kpi: Net Income
Score de crecimiento: 9.0


In [9]:
income_stmt

nding,Revenue,Gross Profit,Operating Income,Net Income
earning_date,,,,
2024-09-30,2646.00,2146.00,365.15,476.57
2023-12-31,2225.00,1794.00,119.97,209.83
2022-12-31,1906.00,1497.00,-161.20,-373.71
2021-12-31,1542.00,1202.00,-411.05,-520.38
2020-12-31,1093.00,740.13,-1174.00,-1166.00
2019-12-31,742.56,500.18,-576.44,-579.65


### Comparar margenes: estado de resultados vs la competencia

In [6]:
peers, score_margins_peers = score_stmt_margins_competitors(
    ticker, 
    income_stmt,
    n_competitors
)

Calculando los margenes de la empresa: PLTR
Calculando los margenes de los competidores
Competidores: ['GOOGL', 'MSFT', 'AMZN', 'MDB', 'SNOW']
Calculando margenes de la competencia...
https://stockanalysis.com/stocks/googl/financials
Request successful
https://stockanalysis.com/stocks/msft/financials
Request successful
https://stockanalysis.com/stocks/amzn/financials
Request successful
https://stockanalysis.com/stocks/mdb/financials
Request successful
https://stockanalysis.com/stocks/snow/financials
Request successful
Score comparación vs competencia: 7.333333333333333


In [7]:
score_stmt_res = score_stmt_res_growth*0.6 + score_margins_peers*0.4
score_stmt_res

8.333333333333332

### Balance General

De aqui obtenemos:

1. Razon corriente
2. Razón corriente "acida"
3. Deuda sobre los activos totales
4. Numero de meses de operación con el dinero en caja

In [14]:
kpis_balance = calculate_kpis_balance_general(ticker, income_stmt_complete)
score_balance = score_balance_general(kpis_balance)

https://stockanalysis.com/stocks/pltr/financials/balance-sheet
Request successful
Gastos operativos mensuales: 111
Dinero en caja: 4565.0
Meses de operación con el dinero en caja: 41.03
Current ratio: 5.67
'Inventory'
Quick ratio: None
Debt ratio: 0.04
Score balance general: 10.0


### Flujo de caja creciente

In [15]:
_, cash_flow = get_financial_data(ticker, "cash_flow")
score_cash_flow_growth = score_growth(cash_flow)

https://stockanalysis.com/stocks/pltr/financials/cash-flow-statement
Request successful
Revisando crecimiento de la compañía...
Analizando kpi: Operating Cash Flow
Analizando kpi: Free Cash Flow
Score de crecimiento: 6.0


### Score salud financiera global

In [17]:
score_stmt_res, score_balance, score_cash_flow_growth

(8.333333333333332, 10.0, 6.0)

In [16]:
score_financial_final = (
    1.*score_stmt_res + 
    1.*score_balance + 
    1.*score_cash_flow_growth
)/3
score_financial_final = round(score_financial_final, 2)
print(f"Score salud financiera final: {score_financial_final}")

Score salud financiera final: 8.11


---